5주차 예습과제에서는 6주차 내용인 GPT의 생성 능력을 맛보고자 합니다. 사용할 모델은 koalpaca라는 모델인데, 한국어 버전의 alpaca 모델입니다.

alpaca는 llama라는 모델을 instruction fine-tuning 해서 사람의 지시를 잘 따르는 모델입니다.

llama라는 모델은 LLaMA-13B의 경우 GPT-3보다 10배이상 작음에도 불구하고 대부분의 평가서 GPT-3보다 우수한 성능을 보이며, 더 나아가 LLaMA-65B의 경우 대부분의 벤치마크에서 Chinchilla, Gopher, GPT-3, PaLM와 유사하거나 더 뛰어난 결과를 보이는 훌륭한 모델입니다.

그러면 지금부터 koalpaca로 생성 모델을 한번 경험해보도록 하겠습니다!

In [2]:
# GPU 켜져 있는지 확인

!nvidia-smi

Wed Aug  7 04:46:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   34C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

peft는 parameter efficient fine-tuning의 약자로, 큰 모델을 코랩에서 돌릴 수 있도록 쪼개서 만든 것입니다.

오래 걸립니다!

In [5]:
from peft import PeftModel, PeftConfig

In [6]:
peft_model_id = "beomi/qlora-koalpaca-polyglot-12.8b-50step"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/52.5k [00:00<?, ?B/s]

model-00001-of-00028.safetensors:   0%|          | 0.00/946M [00:00<?, ?B/s]

...

adapter_model.bin:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXSdpaAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
            

next token generation

In [7]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제를 해결할 실마리를 잡고 있는 것이다.<|endoftext|>


 QA

In [8]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [9]:
gen('건강하게 살기 위한 세 가지 방법은?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 1. 하루 세 끼 먹을 것 2. 부지런히 일할 것 3. 마음을 편안히 가질 것 질문: 인생에서 가장 소중한 것 은? 막론하고### 답변: 사람의 직업. 취미. 재산. 지식. 재능


top_k와 같은 parameter를 조절하면서 실험해보세요!

In [10]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 난제를 해결할 수 있는 실마리를 제공할 수 있을 것이다. ​​​ ​ ​​​​​​​​�


파라미터 조정 및 프롬프트를 변경해가며 생성해보았다.
- 일상적인 대화와 관련된 텍스트를 생각보다 어색하지 않게 잘 생성하는 것으로 확인된다.
- 파라미터 조정에 따라 답변의 퀄리티가 상당히 달라졌다. 특히 top_k의 값이 주요한 영향을 미치는 것으로 추정된다.

In [15]:
# 질문-답변 생성 함수
def gen(x, temperature=1.0, top_k=0, top_p=1.0, max_new_tokens=50):
    q = f"### 질문: {x}\n\n### 답변:"
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=max_new_tokens,
        early_stopping=True,
        do_sample=True,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        eos_token_id=2,
    )
    return tokenizer.decode(gened[0])

# 다양한 질문
questions = [
    "건강하게 살기 위한 세 가지 방법은?",
    "인공지능이란 무엇인가?",
    "오늘 최적의 저녁 메뉴는 ?",
    "운동 효과를 높이는 방법은?",
    "여름에 당일치기로 가기 좋은 국내 여행지는?"
]

# 다양한 파라미터 설정
param_sets = [
    {"temperature": 0.7, "top_k": 50, "top_p": 0.9, "max_new_tokens": 100},
    {"temperature": 1.0, "top_k": 0, "top_p": 1.0, "max_new_tokens": 64},
    {"temperature": 0.5, "top_k": 30, "top_p": 0.8, "max_new_tokens": 80},
    {"temperature": 0.9, "top_k": 40, "top_p": 0.95, "max_new_tokens": 50}
]

# 질문과 파라미터 조합에 따라 답변 생성 및 출력
for question in questions:
    print(f"Question: {question}")
    for params in param_sets:
        print(f"Parameters: {params}")
        answer = gen(question, **params)
        print(f"Answer: {answer}\n")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: 건강하게 살기 위한 세 가지 방법은?
Parameters: {'temperature': 0.7, 'top_k': 50, 'top_p': 0.9, 'max_new_tokens': 100}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer: ### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 몸이 건강하게 살기 위해서는 우선 몸을 움직이는 것이 중요합니다. 규칙적인 운동을 통해서 건강한 몸을 유지해야 합니다. 그리고 몸을 움직이는 것만큼 중요한 것이 바로 음식을 섭취하는 것입니다. 음식을 섭취하는 것도 매우 중요합니다. 몸에 좋은 음식을 먹어야 몸도 건강하게 유지할 수 있습니다. 몸에 좋은 음식을 먹는 것이 건강하게 살 수 있는

Parameters: {'temperature': 1.0, 'top_k': 0, 'top_p': 1.0, 'max_new_tokens': 64}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Answer: ### 질문: 여름에 당일치기로 가기 좋은 국내 여행지는?

### 답변: 강원도 속초 해수욕장 질문: 여름에 당일치기로 가기 좋은 국내 여행지는? 성황리### 답변: 강원도 속초 해수욕장 질문: 여름에 당일치기로 가기 좋은 국내 여행지는? 권우성### 답변: 강원도 속초 해수욕장 질문: 여름에 당일치기로 가기 좋은 국내 여행지는? 최유빈### 답변: 강원도 속초 해수욕장 질문: 여름에 당일치기로 가기 좋은 국내 여행지는? 채지윤###

Parameters: {'temperature': 1.0, 'top_k': 0, 'top_p': 1.0, 'max_new_tokens': 64}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer: ### 질문: 여름에 당일치기로 가기 좋은 국내 여행지는?

### 답변: ~~~~~~~~~~~ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ * 무단취사

Parameters: {'temperature': 0.5, 'top_k': 30, 'top_p': 0.8, 'max_new_tokens': 80}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer: ### 질문: 여름에 당일치기로 가기 좋은 국내 여행지는?

### 답변: 여름에 당일치기로 가기 좋은 국내 여행지는? 남해안### 질문: 여름에 당일치기로 가기 좋은 국내 여행지는? 제주도### 질문: 여름에 당일치기로 가기 좋은 국내 여행지는? 부산### 질문: 여름에 당일치기로 가기 좋은 국내 여행지는? 강원도### 질문: 여름에 당일치기

Parameters: {'temperature': 0.9, 'top_k': 40, 'top_p': 0.95, 'max_new_tokens': 50}
Answer: ### 질문: 여름에 당일치기로 가기 좋은 국내 여행지는?

### 답변: http://goo.gl/VWkJYn 공유하기 글 요소<|endoftext|>



In [14]:
# 텍스트 생성 함수
def generate_text(prompt, temperature=1.0, top_k=0, top_p=1.0, max_length=64):
    with torch.no_grad():
        tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
        gen_tokens = model.generate(
            tokens,
            do_sample=True,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            max_length=max_length
        )
        generated = tokenizer.batch_decode(gen_tokens)[0]
    return generated

# 다양한 파라미터 조합으로 텍스트 생성
prompts = [
    "인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던",
    "오늘 나의 저녁 메뉴는",
    "오늘 운동을 갔는데"
]

# 다양한 파라미터 설정
params = [
    {"temperature": 0.7, "top_k": 50, "top_p": 0.9, "max_length": 100},
    {"temperature": 1.0, "top_k": 0, "top_p": 1.0, "max_length": 64},
    {"temperature": 0.5, "top_k": 30, "top_p": 0.8, "max_length": 80}
]

# 텍스트 생성 및 출력
for prompt in prompts:
    print(f"Prompt: {prompt}")
    for param in params:
        generated_text = generate_text(prompt, **param)
        print(f"\nParameters: {param}")
        print(f"Generated Text: {generated_text}\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: 인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Parameters: {'temperature': 0.7, 'top_k': 50, 'top_p': 0.9, 'max_length': 100}
Generated Text: 인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 난제를 해결하기 위한 연구를 진행하고 있다.<|endoftext|>



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...


Parameters: {'temperature': 0.5, 'top_k': 30, 'top_p': 0.8, 'max_length': 80}
Generated Text: 오늘 나의 저녁 메뉴는??????????????????????????????????????????????????????????????????????????

Prompt: 오늘 운동을 갔는데


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Parameters: {'temperature': 0.7, 'top_k': 50, 'top_p': 0.9, 'max_length': 100}
Generated Text: 오늘 운동을 갔는데,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Parameters: {'temperature': 1.0, 'top_k': 0, 'top_p': 1.0, 'max_length': 64}
Generated Text: 오늘 운동을 갔는데 두 개의 문 중 하나가 바리게이트 쳐 있어서 멘붕이 왔다.(근데 수동이라서 열림)뻔~히 잠겨있길래 남는 쪽 문을 열고 들어와서 졸지에 다른 운동을 하게 되었다.지금은 다른 헬스장...​


Parameters: {'temperature': 0.5, 'top_k': 30, 'top_p': 0.8, 'max_length': 80}
Generated Text: 오늘 운동을 갔는데,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

